# **SPX & VIX Options Data Collection**

NOTE: The OptyX approach predicts the VIX on an intraday scale by using just a subset of the most liquid options. The process of obtaining this specific list of options is detailed within this notebook.  

The methodology is as follows: 

*All options, at any given point in time, must satisfy the following criteria:*

*1.) Expiration date between 23 and 37 days in the future*

*2.) Bid and ask greater than zero*

*3.) Strike within .4% of spot at beginning of day.*

# __1.) Prepping 30 min SPX options data: *Years 2017 through 2019*__

### __1.1.) Import data and initial data exploration__

In [ ]:
#Importing packages

import pandas as pd 
import numpy as np 
from pathlib import Path 
import hvplot.pandas
import datetime as dt 

In [ ]:
#Importing spx_17_19_30min.csv

spx_17_19 = pd.read_csv(
    Path("../Resources/spx_17_19_30min.csv"),  #NOTE: Too large for Gitbub. Skip to section 3.
    index_col='QUOTE_READTIME',
    parse_dates=True,
    infer_datetime_format=True,
    low_memory=False
)

spx_17_19

In [ ]:
spx_17_19.columns

In [ ]:
#Check date of expirations, ensuring Fridays only

freq = spx_17_19['EXPIRE_DATE']
freq.value_counts().sort_values(ascending=False).iloc[50:100]

### __1.1.) Selecting only contracts between 27 and 32 days until expiration__

In [ ]:
#Selecting only contracts with expiration dates between 23 and 37 days in the future

spx_17_19 = spx_17_19.loc[(spx_17_19['DTE'] <= 32) & (spx_17_19['DTE'] >= 27)]

In [ ]:
spx_17_19

In [ ]:
# Checking frequency of DTE values

test1 = spx_17_19.DTE.sort_values()
freq = pd.DataFrame(test1)
freq.value_counts().sort_values(ascending=False).iloc[0:50]

### __1.2.) Forming Contract ID__

In [ ]:
spx_17_19['CONTRACT'] = spx_17_19['STRIKE'].astype(str) + spx_17_19['EXPIRE_DATE']

### **1.3.) Inception Filtering Process (selecting only contracts the meet certain criteria at the start of the day)**

In [ ]:
spx_17_19 = spx_17_19.reset_index()

In [ ]:
spx_17_19['QUOTE_READTIME'] = spx_17_19['QUOTE_READTIME'].astype(str)

In [ ]:
#INCEPTION, Step 1. Filter by selecting only contracts at the start of the day.

start_of_day = spx_17_19.loc[spx_17_19['QUOTE_READTIME'].str.contains('09:30')]
start_of_day   

In [ ]:
#INCEPTION, Step 2. Filter start-of-day contracts by selecting only those within .4% of strike 

filt = start_of_day.loc[(start_of_day['STRIKE_DISTANCE_PCT'] <= 0.004)] 
filt

In [ ]:
#INCEPTION, Step 3. Form a list of the contract IDs from the filtered DataFrame.

criteria_list = list(filt['CONTRACT'])

In [ ]:
#INCEPTION, Step 4. Get the length of this list.

len(criteria_list)

In [ ]:
#INCEPTION, Step 5. Overlay this list on top of original DataFrame, filtering to keep only these contract IDs that are in the list.

spx_17_19_filt = spx_17_19[spx_17_19["CONTRACT"].isin(criteria_list)]

In [ ]:
spx_17_19_filt

### **1.4.) Changing data types**

In [ ]:
spx_17_19_filt['C_IV'] = pd.to_numeric(spx_17_19_filt['C_IV'], errors = 'coerce')
spx_17_19_filt['P_IV'] = pd.to_numeric(spx_17_19_filt['P_IV'], errors = 'coerce')
spx_17_19_filt['C_BID'] = pd.to_numeric(spx_17_19_filt['C_BID'], errors = 'coerce')
spx_17_19_filt['C_ASK'] = pd.to_numeric(spx_17_19_filt['C_ASK'], errors = 'coerce')

In [ ]:
#checking datatype
spx_17_19_filt.dtypes

### __1.5.) Selecting only contracts that have both a bid and ask greater than 0__

In [ ]:
spx_17_19_filt2 = spx_17_19_filt.loc[(spx_17_19_filt[' [P_BID]'] > 0) & (spx_17_19_filt['P_ASK'] > 0)]
spx_17_19_filt2 = spx_17_19_filt2.loc[(spx_17_19_filt2['C_BID'] > 0) & (spx_17_19_filt2['C_ASK'] > 0)]

In [ ]:
spx_17_19_filt2['QUOTE_READTIME'] = pd.to_datetime(spx_17_19_filt2['QUOTE_READTIME'])

In [ ]:
spx_17_19_filt2 = spx_17_19_filt2.set_index('QUOTE_READTIME')

In [ ]:
#Plotting for select time periods, grouping by contracts 

dftest = spx_17_19_filt2.loc["2022-01-03 9:30:00":"2022-06-30 16:00:00"]
print(spx_17_19_filt2.iloc[3000])

dftest.groupby('CONTRACT')['STRIKE'].plot(figsize=(30,12))
dftest.groupby('CONTRACT')['UNDERLYING_LAST'].plot(figsize=(30,12))

In [ ]:
spx_17_19_filt2.to_csv("../Resources/spx_17_19_prep1_V1.csv", index="QUOTE_READTIME")

# __2.) Prepping 30 min SPX options data: *Years 2020 through 2022*__

### __2.0.) Import data and initial data exploration__

In [ ]:
import pandas as pd 
import numpy as np 
from pathlib import Path 
import hvplot.pandas
import datetime as dt 

In [ ]:
#Importing spx_20_22_30min.csv

spx_20_22 = pd.read_csv(
    Path("../Resources/spx_20_22_30min.csv"),   #NOTE: Too large for Gitbub. Skip to section 3.
    index_col='QUOTE_READTIME',
    parse_dates=True,
    infer_datetime_format=True,
    low_memory=False
)

spx_20_22

In [ ]:
spx_20_22.columns

In [ ]:
#Check date of expirations, ensuring Fridays only

freq = spx_20_22['EXPIRE_DATE']
freq.value_counts().sort_values(ascending=False).iloc[50:100]

### __2.1.) Selecting only contracts between 27 and 32 days until expiration__

In [ ]:
#Selecting only contracts with expiration dates between 23 and 37 days in the future

spx_20_22 = spx_20_22.loc[(spx_20_22['DTE'] <= 32) & (spx_20_22['DTE'] >= 27)]

In [ ]:
spx_20_22

In [ ]:
# Checking frequency of DTE values

test1 = spx_20_22.DTE.sort_values()
freq = pd.DataFrame(test1)
freq.value_counts().sort_values(ascending=False).iloc[0:50]

### __2.2.) Forming Contract ID__

In [ ]:
spx_20_22['CONTRACT'] = spx_20_22['STRIKE'].astype(str) + spx_20_22['EXPIRE_DATE']

### **2.3.) Inception Filtering Process (selecting only contracts the meet certain criteria at the start of the day)**

In [ ]:
spx_20_22 = spx_20_22.reset_index()

In [ ]:
#Converting QUOTE_READTIME to string to enable parsing apart 

spx_20_22['QUOTE_READTIME'] = spx_20_22['QUOTE_READTIME'].astype(str)

In [ ]:
#INCEPTION, Step 1. Filter by selecting only contracts at the start of the day.

start_of_day = spx_20_22.loc[spx_20_22['QUOTE_READTIME'].str.contains('09:30')]
start_of_day   

In [ ]:
#INCEPTION, Step 2. Filter start-of-day contracts by selecting only those within .4% of strike.

filt = start_of_day.loc[(start_of_day['STRIKE_DISTANCE_PCT'] <= 0.004)] 
filt

In [ ]:
#INCEPTION, Step 3. Form a list of the contract IDs from the filtered DataFrame.

criteria_list = list(filt['CONTRACT'])

In [ ]:
#INCEPTION, Step 4. Get the length of this list.

len(criteria_list)

In [ ]:
#INCEPTION, Step 5. Overlay this list on top of original DataFrame, filtering to keep only these contract IDs that are in the list.

spx_20_22_filt = spx_20_22[spx_20_22["CONTRACT"].isin(criteria_list)]

In [ ]:
spx_20_22_filt

### **2.4.) Changing data types**

In [ ]:
spx_20_22_filt['C_IV'] = pd.to_numeric(spx_20_22_filt['C_IV'], errors = 'coerce')
spx_20_22_filt['P_IV'] = pd.to_numeric(spx_20_22_filt['P_IV'], errors = 'coerce')
spx_20_22_filt['C_BID'] = pd.to_numeric(spx_20_22_filt['C_BID'], errors = 'coerce')
spx_20_22_filt['C_ASK'] = pd.to_numeric(spx_20_22_filt['C_ASK'], errors = 'coerce')

spx_20_22_filt['C_DELTA'] = pd.to_numeric(spx_20_22_filt['C_DELTA'], errors = 'coerce')
spx_20_22_filt['C_GAMMA'] = pd.to_numeric(spx_20_22_filt['C_GAMMA'], errors = 'coerce')
spx_20_22_filt['C_VEGA'] = pd.to_numeric(spx_20_22_filt['C_VEGA'], errors = 'coerce')
spx_20_22_filt['C_THETA'] = pd.to_numeric(spx_20_22_filt['C_THETA'], errors = 'coerce')

spx_20_22_filt['C_RHO'] = pd.to_numeric(spx_20_22_filt['C_RHO'], errors = 'coerce')
spx_20_22_filt['C_LAST'] = pd.to_numeric(spx_20_22_filt['C_LAST'], errors = 'coerce')
spx_20_22_filt[' [P_BID]'] = pd.to_numeric(spx_20_22_filt[' [P_BID]'], errors = 'coerce')
spx_20_22_filt['P_ASK'] = pd.to_numeric(spx_20_22_filt['P_ASK'], errors = 'coerce')

spx_20_22_filt['P_LAST'] = pd.to_numeric(spx_20_22_filt['P_LAST'], errors = 'coerce')
spx_20_22_filt['P_DELTA'] = pd.to_numeric(spx_20_22_filt['P_DELTA'], errors = 'coerce')
spx_20_22_filt['P_GAMMA'] = pd.to_numeric(spx_20_22_filt['P_GAMMA'], errors = 'coerce')
spx_20_22_filt['P_VEGA'] = pd.to_numeric(spx_20_22_filt['P_VEGA'], errors = 'coerce')

spx_20_22_filt['P_THETA'] = pd.to_numeric(spx_20_22_filt['P_THETA'], errors = 'coerce')
spx_20_22_filt['P_RHO'] = pd.to_numeric(spx_20_22_filt['P_RHO'], errors = 'coerce')

In [ ]:
spx_20_22_filt.dtypes

### __2.5.) Selecting only contracts that have both a bid and ask greater than 0__

In [ ]:
spx_20_22_filt2 = spx_20_22_filt.loc[(spx_20_22_filt[' [P_BID]'] > 0) & (spx_20_22_filt['P_ASK'] > 0)]
spx_20_22_filt2 = spx_20_22_filt2.loc[(spx_20_22_filt2['C_BID'] > 0) & (spx_20_22_filt2['C_ASK'] > 0)]

In [ ]:
spx_20_22_filt2['QUOTE_READTIME'] = pd.to_datetime(spx_20_22_filt2['QUOTE_READTIME'])

In [ ]:
spx_20_22_filt2 = spx_20_22_filt2.set_index('QUOTE_READTIME')

In [ ]:
spx_20_22_filt2

In [ ]:
spx_20_22_filt2.to_csv("../Resources/spx_20_22_prep1_V1.csv", index="QUOTE_READTIME")

# __3.) Combining SPX Options Data: *Years 2017 through 2019* with *Years 2020 through 2022*__

### __3.1.) Imports__

In [ ]:
#Years 2017 through 2019, spx_17_19_prep1_V1.csv

spx_17_19_ready = pd.read_csv(
    Path("../Resources/spx_17_19_prep1_V1.csv"),
    index_col='QUOTE_READTIME',
    parse_dates=True,
    infer_datetime_format=True,
    low_memory=False
)

spx_17_19_ready 

In [ ]:
# Years 2020 through 2022, spx_20_22_prep1_V1.csv

spx_20_22_ready = pd.read_csv(
    Path("../Resources/spx_20_22_prep1_V1.csv"),
    index_col='QUOTE_READTIME',
    parse_dates=True,
    infer_datetime_format=True,
    low_memory=False
)

spx_20_22_ready

### __3.2.) Concatinating both periods__

In [ ]:
spx_17_22 = pd.concat([spx_17_19_ready, spx_20_22_ready])

In [ ]:
spx_17_22

In [ ]:
spx_17_22 = spx_17_22.dropna()

In [ ]:
spx_17_22

In [ ]:
spx_17_22.to_csv("../Resources/spx_17_22.csv", index="QUOTE_READTIME")

# __4.) Prepping 30 min VIX Data: *Years 2017 through 2022*__

### __4.1.) Import and Filtering__

In [ ]:
# Importing VIX data, years 2017 through 2022

vix = pd.read_csv(
    Path("../Resources/vix30min.csv"),
    index_col='QUOTE_READTIME',
    parse_dates=True,
    infer_datetime_format=True,
    low_memory=False
)

vix

In [ ]:
vix = vix[['VIX']]

In [ ]:
vix

In [ ]:
vix.to_csv("../Resources/vix_final_30.csv", index="QUOTE_READTIME")

# __5.) Calculating weights for each strike price__

In [ ]:
import pandas as pd 
import numpy as np 
from pathlib import Path 
import hvplot.pandas
import datetime as dt 

In [ ]:
# Years 2020 through 2022

vix_spx = pd.read_csv(
    Path("../Resources/spx_17_22.csv"),
    index_col='QUOTE_READTIME',
    parse_dates=True,
    infer_datetime_format=True,
    low_memory=False
)

vix_spx

In [ ]:
#Selecting only STRIKE column and placing into a new DataFrame, "strikes"

strikes = vix_spx[['STRIKE']]
strikes

In [ ]:
#Calculating each strike's impact on the VIX with (delta K / K^2)

strikes['Weights'] = (((strikes.shift(-1)) - (strikes.shift(1)))/2) / (strikes**2)

In [ ]:
#Scaling so weights are in interpretable format

strikes['Weights'] = strikes['Weights']*1e6
strikes

In [ ]:
#Forward filling all NaNs and then backfilling just the first NaN

strikes = strikes.ffill()
strikes = strikes.bfill()

In [ ]:
strikes.head(50)I 

In [ ]:
strikes = strikes.sort_values(by=["QUOTE_READTIME", "STRIKE"])

In [ ]:
strikes

In [ ]:
#Merging strikes DataFrame (contains weights for each strike) with vix_spx

vix_spx = vix_spx.merge(strikes, on=['QUOTE_READTIME', 'STRIKE'])

In [ ]:
vix_spx.reset_index(inplace=True)
vix_spx.head()

In [ ]:
vix_spx = vix_spx.drop(columns=['EXPIRE_DATE'])
vix_spx = vix_spx.drop(columns=['CONTRACT'])

In [ ]:
vix_spx

In [ ]:
vix_spx.to_csv('../Resources/vix_spx_weighted.csv', index=None)

# __5.) Calculating weighted averages for each feature__

In [ ]:
#Importing vix_spx_weighted.csv

vix_spx = pd.read_csv(
    Path("../Resources/vix_spx_weighted.csv"),
    index_col='QUOTE_READTIME',
    parse_dates=True,
    infer_datetime_format=True,
    low_memory=False
)
vix_spx

In [ ]:
#Dropping columns note needed for LSTM

vix_spx = vix_spx.drop(columns=['C_LAST', 'P_LAST'])

In [ ]:
#Calculating premiums for both calls and puts

vix_spx['C_Premium'] = (vix_spx['C_BID'] + vix_spx['C_ASK']) / 2
vix_spx['P_Premium'] = (vix_spx[' [P_BID]'] + vix_spx['P_ASK']) / 2

vix_spx.head()

In [ ]:
#Dropping columns note needed for LSTM

vix_spx = vix_spx.drop(columns=['C_BID', 'C_ASK', ' [P_BID]', 'P_ASK'])

In [ ]:
col_count = vix_spx.shape[1]
col_count

In [ ]:
vix_spx.iloc[:, 1]

In [ ]:
vix_spx.reset_index(inplace=True)

In [ ]:
#Creating aggregation function for calculating weighted means. Groups by QUOTE_READTIME when called.

def grouped_weighted_avg(values, weights, by):
    return (values * weights).groupby(by).sum() / weights.groupby(by).sum()

In [ ]:
#Looping through each column to calculate weighted means for each timestamp

df_dict = {}

for i in range(1, (col_count)+1):
    col_name = vix_spx.columns[i]
    series = grouped_weighted_avg(vix_spx.iloc[:, i], vix_spx["Weights"], vix_spx["QUOTE_READTIME"])
    df_i = pd.DataFrame(data=[series], index=[col_name])
    df_i = df_i.T
    df_i.index = pd.to_datetime(df_i.index)
    df_dict[col_name] = series

In [ ]:
#Creating a DataFrame from the dictionary created in the for_loop directly above

vix_spx_w = pd.DataFrame(data=df_dict)

In [ ]:
vix_spx_w = vix_spx_w.drop(columns=['Weights'])

In [ ]:
vix_spx_w.columns

In [ ]:
vix_spx_w

In [ ]:
vix_spx_w.to_csv("../Resources/vix_spx_w.csv")

# __7.) Combining SPX Options Data with VIX Data: *Years 2017 through 2022*__

### __7.1) Imports__

In [ ]:
import pandas as pd 
import numpy as np 
from pathlib import Path 
import hvplot.pandas
import datetime as dt 
from functools import reduce

In [ ]:
# Importing VIX data, years 2017 through 2022

vix_final = pd.read_csv(
    Path("../Resources/vix_final_30.csv"),
    index_col='QUOTE_READTIME',
    parse_dates=True,
    infer_datetime_format=True,
    low_memory=False
)

vix_final

In [ ]:
# Importing SPX data, years 2017 through 2022

spx_final = pd.read_csv(
    Path("../Resources/vix_spx_w.csv"),
    index_col='QUOTE_READTIME',
    parse_dates=True,
    infer_datetime_format=True,
    low_memory=False
)

spx_final

### __7.2.) Concatinating VIX with SPX data, 2017 through 2022__

In [ ]:
vix_spx = pd.merge_asof(spx_final, vix_final, on='QUOTE_READTIME')
vix_spx.isna().sum().sum()

In [ ]:
vix_spx = vix_spx.set_index('QUOTE_READTIME')
vix_spx

In [ ]:
#Re-arranging column order

VIX_col = vix_spx['VIX']
vix_spx = vix_spx.drop(columns=['VIX'])
vix_spx.insert(loc=0, column='VIX', value=VIX_col)
vix_spx

In [ ]:
vix_spx.to_csv("../Resources/vix_spx_ready.csv")